Explications et commentaires sur les méthodes utilisées

D'une part, il est important de préciser que nous avons exclus les fonds avec historiques incomplets (6 fonds thématiques sur 40) Approche conservative qui permet de ne pas avoir à traiter les valeurs manquantes des fonds qui ont été construits après le debut de notre jeu de donnée. 
Il s'agit d'une simplification, qui nous permet de traiter tous les fonds de façon homogène. 
En effet pour certains modèles, la taille des windows, ou autre pourrait ne pas convenir pour ces fonds.
L'implémentation pour ces fonds reste toutefois à portée de vue. 


L'objectif du projet est de réduire au maximum l'erreur de tracking entre un portefeuille thématique donné et un portefeuille sectoriel. Le programme de minimisation est le suivant : 

$$\omega^*_{t,\text{sectoriel}} = \underset{\omega_{t,\text{sectoriel}}}{\arg\min} L\left(r_t, \hat{r}_t\right)$$

Sous contraintes : 

- La somme des poids sectoriels doit être égale à 1 (portefeuille totalement investi) :   $$\sum_{i=1}^{N} \omega_i = 1$$
  où $N$ est le nombre de secteurs.

- Les poids sectoriels doivent être compris entre 0 et 1 (pas de ventes à découvert, ni de levier) :
   $$0 \leq \omega_i \leq 1 \quad \forall i \in [1, N]$$


où :
- $\omega^*_{t,\text{sectoriel}}$ : poids sectoriels optimaux à l'instant $t$
- $L(\cdot, \cdot)$ : fonction de perte (loss function)
- $r_t$ = $r_{t,\Delta T}(V_p(P_{t,\text{thématique}}))$ : rendement sur la période $\Delta T$ du portefeuille thématique $P_{t,\text{thématique}}$ à l'instant $t$
- $\hat{r}_t$ = $r_{t,\Delta T}(V_p(P_{t,\text{sectoriel}}))$ : rendement sur la période $\Delta T$ du portefeuille sectoriel $P_{t,\text{sectoriel}}$ à l'instant $t$
- $V_p(\cdot)$ : fonction de valeur du portefeuille


## 1er modèle : Optimisation Convexe sous contrainte : MSE, Sans rebalancement

Ici, le choix de la fonction de perte (ou tracking error dans notre cas) est crutial. Nos premiers modèles se construisent à partir de La fonction de perte quadratique, également appelée **Mean Squared Error (MSE)**, est définie comme suit :

$$L(r_t, \hat{r}_t) = \frac{1}{T} \sum_{t=1}^{T} (r_t - \hat{r}_t)^2$$

où :
- $r_t$ : rendement réel du portefeuille thématique à l'instant $t$
- $\hat{r}_t$ : rendement prédit du portefeuille sectoriel répliqué à l'instant $t$ (i.e., $\hat{r}_t = X_t \cdot \omega$)
- $T$ : nombre total d'observations temporelles

Le **RMSE (Root Mean Squared Error)** est la racine carrée du MSE correspondra finalement à notre tracking error:

$$\text{RMSE} = \sqrt{\frac{1}{T} \sum_{t=1}^{T} (r_t - \hat{r}_t)^2}$$



Cette fonction de perte est en effet la plus adaptée pour caractériser le tracking indiciel. 
The RMSE criterion is used as,
according to Beasley et al. (2003), it is one of the most effective
measurements for addressing this type of index-tracking problems.
Using only the variance of {(rt  Rt)|t = 1, ... , T} as a tracking error
measure (see Connor & Leland, 1995; Franks, 1992; Larsen &
Resnick, 1998; Rohweder, 1998) could potentially lead to erroneous
results, as the tracking portfolios would constantly underperform
the index because they would ignore the proportion (rt - Rt). for example, let M > 0 be a constant; when rt = Rt – M for all t the tracking portfolio has a zero tracking error, but will always underperform
the benchmark index.



Notre fonction compose_monthly_returns permet de retrouver à partir des rendements journaliers, le rendement sur la période pour chaque portefeuille sectoriel et thématique.

à noter que la fonction de valeur du portefeuille ici n'est pas nécessaire, comme les données fournies sont uniquement exprimées en retours, et non pas en prix. 
Ainsi, nos travaux s'inscrivent dans une litterature sur l'index tracking à partir des retours : "The return-based formulations measure the tracking error as
a function of the differences between the historical returns of
the tracking portfolio and the index (cf., e.g., Andriosopoulos &
Nomikos, 2014)." (Optimal construction and rebalancing of index-tracking portfolios
O. Strub, P. Baumann)

Various mathematical programing formulations for the indextracking problem have been proposed in the literature. These
formulations determine a tracking portfolio by minimizing an
objective function, referred to as tracking error, that measures
the difference between the historical performance of the tracking
portfolio and the index. 





Désavantages de cette méthode : 

Les résultats sont peu convainquant, tracking erreur assez élevée : Plusieurs réponses pour celà. D'une part, le modèle ne comprend pas un rebalancement de portefeuille chaque mois. Cela evidemment influe sur la performance du tracking (on incorpore les données de test passées dans les données d'entrainement chaque mois passé.)
De plus on peut avoir un certain surentrainement avec les valeurs passés lointaines. 

## Modèle 2 : Optimisation sous contrainte : MSE, Rebalancement (expanding window)

Le deuxième modèle propose une amélioration du premier modèle en implémentant un système de rebalancement qui :
1. Utilise une fenêtre d'entraînement qui s'étend (expanding window) au lieu de la fenètre fixe "in-sample"
2. Recalcule les poids optimaux chaque mois en incorporant les données de test passées dans l'entraînement
3. Permet d'améliorer progressivement le modèle au fur et à mesure que de nouvelles données deviennent disponibles

Avantages :
- Utilise toutes les données disponibles jusqu'à chaque point dans le temps
- Ajuste dynamiquement les poids en fonction de l'évolution des données
- Plus réaliste pour un contexte de trading réel

## Modèle 3 : Optimisation sous contrainte Fenêtre glissante (Sliding Window) avec rebalancement

 Le troisième modèle introduit une nouvelle méthode d’optimisation basée sur une fenêtre glissante (« sliding window ») pour l’entraînement. À chaque période de rebalancement, on n’utilise que les observations des N derniers mois (fenêtre de taille fixe) pour recalculer les poids optimaux du portefeuille, au lieu de considérer toutes les données disponibles depuis le début. Cette approche est plus réactive aux récents changements de marché et permet de capturer de potentielles modifications dans la relation entre les secteurs et le benchmark thématique.
 
 **Avantages principaux de la fenêtre glissante :**
 - Le modèle se concentre sur les données récentes, ce qui lui permet de s’adapter plus rapidement aux nouvelles dynamiques de marché ou à des ruptures structurelles.
 - La limitation de la fenêtre réduit les risques d’overfitting sur des données anciennes qui ne sont plus représentatives de la période actuelle.
 - La stabilité du modèle est accrue, car les poids du portefeuille ne sont influencés que par une période limitée et pertinente dans le temps.
 - Cette méthodologie est couramment utilisée dans la pratique du suivi d'indice, surtout dans des environnements où les corrélations et les tendances sectorielles peuvent évoluer rapidement.


## 4. Modèle Ridge avec rebalancement et sliding window

Afin de pallier le problème de multicolinéarité entre les variables explicatives (par exemple, les rendements sectoriels corrélés),
on implémente une régression Ridge, qui consiste à ajouter une pénalisation L2 sur la somme des carrés des coefficients. 
Cette régularisation permet de stabiliser l'estimation des poids du portefeuille tout en conservant une bonne capacité de suivi.

À chaque période de rebalancement, les poids sont recalculés via la résolution du problème d'optimisation suivant :

$$
\begin{align*}
\min_{\omega}\quad & \| y - X \omega \|_2^2 + \alpha \|\omega\|_2^2 \\
\text{sous contraintes} \quad & \sum_{i=1}^{N} \omega_i = 1 \\
& \omega_i \geq 0 \quad \forall i = 1, \ldots, N
\end{align*}
$$

où :
- $y$ est le vecteur des rendements du benchmark thématique à chaque date de la fenêtre d'entraînement ;
- $X$ est la matrice des rendements sectoriels sur la même période ;
- $\omega$ est le vecteur de poids cherché ;
- $\alpha$ est le paramètre de régularisation Ridge (>0).

Ce modèle permet une meilleure robustesse des poids en présence de secteurs corrélés et favorise une solution moins sensible 
au bruit ou à l'overfitting.

Ce nouveau modèle de réplication s'appuie sur une approche d'optimisation dite "mean-variance tracking". Contrairement à une simple régression des rendements, il cherche explicitement à minimiser l'écart de variance entre les rendements du portefeuille thématique réel et ceux du portefeuille répliqué construit à partir des secteurs. Pour cela, le modèle estime la matrice de covariance des rendements sectoriels ainsi que leur covariance croisée avec la cible thématique, puis résout un problème d'optimisation quadratique sous contraintes (pondérations positives et somme à 1). L'objectif est d'obtenir une combinaison de secteurs qui imite au plus près le profil de risque et de rendement de l'indice thématique, tout en assurant une robustesse accrue face à la colinéarité des secteurs et à la volatilité des séries temporelles.


## Modèle 4 : Algorithme génétique avec rebalancement et sliding window

Ce modèle s'appuie sur une approche d'optimisation métaheuristique basée sur les algorithmes génétiques pour résoudre le problème de réplication de portefeuille. Contrairement aux méthodes d'optimisation convexe précédentes qui cherchent un optimum global garanti, l'algorithme génétique explore l'espace des solutions de manière stochastique en simulant un processus d'évolution biologique. Cette approche est particulièrement adaptée aux problèmes d'optimisation de portefeuille avec contraintes non-linéaires ou lorsque la fonction objectif présente des optima locaux multiples, comme l'ont démontré Andriosopoulos et Nomikos dans leur étude sur la réplication de performance d'indices énergétiques (2014).

L'algorithme génétique opère sur une population de solutions candidates (vecteurs de poids), où chaque individu représente un portefeuille possible. Le processus itératif comprend trois opérations principales : la sélection, le croisement et la mutation. À chaque génération, les individus sont évalués selon leur fonction de fitness, définie ici comme l'inverse du tracking error (RMSE). Les meilleurs individus sont sélectionnés selon un mécanisme de tournoi, puis subissent des opérations de croisement (combinaison de deux solutions parentes) et de mutation (perturbation aléatoire) pour générer une nouvelle génération. Les contraintes de somme égale à 1 et de positivité des poids sont respectées via une normalisation après chaque opération génétique.

La fonction de fitness minimisée est :

$$f(\omega) = \text{RMSE}(\omega) = \sqrt{\frac{1}{T} \sum_{t=1}^{T} (y_t - X_t \omega)^2}$$

où chaque individu (vecteur de poids $\omega$) est évalué sur sa capacité à minimiser l'erreur de tracking sur la fenêtre d'entraînement. Les opérateurs génétiques utilisés sont un croisement de type blend (combinaison linéaire des parents) et une mutation gaussienne suivie d'une renormalisation pour garantir le respect des contraintes.

**Avantages de cette approche :**

- Exploration globale de l'espace des solutions : contrairement aux méthodes déterministes qui peuvent converger vers des optima locaux, l'algorithme génétique explore simultanément plusieurs régions de l'espace de recherche, ce qui permet de découvrir des solutions potentiellement meilleures dans des problèmes non-convexes ou avec des contraintes complexes.

- Robustesse face aux optima locaux : la nature stochastique de l'algorithme et le maintien d'une population diversifiée permettent d'éviter la convergence prématurée vers des solutions sous-optimales, particulièrement utile lorsque la fonction objectif présente plusieurs minima locaux.

- Flexibilité dans la formulation du problème : l'algorithme génétique peut facilement intégrer des contraintes additionnelles ou des objectifs multiples sans nécessiter de reformulation mathématique complexe, ce qui le rend adaptatif à différentes configurations de contraintes de portefeuille.

- Capacité à gérer des espaces de recherche de grande dimension : même avec un nombre élevé de secteurs, l'algorithme génétique maintient une exploration efficace grâce à ses mécanismes de diversité génétique, comme démontré dans l'application aux marchés britannique, américain et brésilien par Andriosopoulos et Nomikos (2014).

- Pas d'hypothèse sur la convexité : contrairement aux méthodes d'optimisation convexe, l'algorithme génétique ne nécessite pas que la fonction objectif soit convexe ou différentiable, ce qui le rend applicable à des problèmes plus complexes où les méthodes classiques peuvent échouer.

## Modèle 5 : Mean-Variance Tracking avec rebalancement et sliding window

Ce modèle de réplication se différencie par le choix de la fonction de perte.  
Contrairement aux modèles précédents qui minimisent directement l'erreur quadratique moyenne des rendements, ce modèle cherche explicitement à minimiser la variance de l'erreur de tracking entre les rendements du portefeuille thématique réel et ceux du portefeuille répliqué construit à partir des secteurs. Pour cela, le modèle estime la matrice de covariance des rendements sectoriels ainsi que leur covariance croisée avec la cible thématique, puis résout un problème d'optimisation quadratique sous contraintes (pondérations positives et somme à 1). 

L'objectif mathématique est de minimiser la variance de l'erreur de tracking :

$$\text{Var}(y - X \omega) = \omega^T \Sigma_X \omega - 2 \omega^T \Sigma_{Xy} + \text{Var}(y)$$

où :
- $\Sigma_X = \text{Cov}(X)$ est la matrice de covariance des rendements sectoriels, estimée via l'estimateur Ledoit-Wolf pour améliorer la robustesse avec des échantillons de taille limitée
- $\Sigma_{Xy} = \text{Cov}(X, y)$ est le vecteur de covariance croisée entre les secteurs et le portefeuille thématique
- $\text{Var}(y)$ est la variance du portefeuille thématique (constante dans l'optimisation)

Le programme d'optimisation résolu est :

$$\omega^* = \underset{\omega}{\arg\min} \left\{ \omega^T \Sigma_X \omega - 2 \omega^T \Sigma_{Xy} \right\}$$

sous les contraintes habituelles de somme égale à 1 et de positivité des poids.

**Avantages de cette approche :**

- Prise en compte explicite de la structure de risque : le modèle optimise directement la variance de l'erreur de tracking, ce qui permet de répliquer non seulement les rendements moyens mais aussi le profil de risque du portefeuille thématique.

- Robustesse face à la multicolinéarité : l'utilisation de l'estimateur Ledoit-Wolf régularise la matrice de covariance, ce qui améliore la stabilité des poids estimés lorsque les secteurs sont fortement corrélés entre eux.

- Stabilité accrue des poids : l'optimisation basée sur la variance tend à produire des poids plus stables d'une période à l'autre, ce qui peut réduire les coûts de transaction associés aux rébalancements fréquents.

- Cohérence théorique : cette approche s'inscrit dans le cadre de la théorie moderne du portefeuille (Markowitz), en minimisant le risque de tracking plutôt que simplement l'erreur moyenne, ce qui est plus adapté pour le suivi d'indice.

- Adaptation aux corrélations : le modèle utilise explicitement les corrélations entre secteurs et entre secteurs et portefeuille thématique pour construire une combinaison plus cohérente et robuste.